In [65]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [38]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'
stats = pd.read_csv(path + '/all_files_stats.csv')
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
#display(files)
listcol= files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
listcol.filename = listcol.filename.str.split('_', expand=True)[0]
listcol.index = listcol['filename']
listcol
listcolt = listcol
listcolt #= listcolt[(listcolt['filename']=='TVEpisode') | (listcolt['filename']=='MusicAlbum')]
listcolt#[:5]

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [41]:
overview = pd.DataFrame()
for index in listcolt.filename:
    round = pd.DataFrame()
    print(index)
    print(round)
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
    for file in files:
        # read file
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        numrows= df.shape[0]
        list_col = list(df.columns)
        round = round.append({'Class':index,'filename':file,'num_rows':numrows,'columns':list_col}, ignore_index=True)
    liste = set(listcolt.loc[index]['listofcolumns'])
    round = round.explode('columns')
    round = round[round['columns'].isin(liste)]
    overview = overview.append(round)
    display(overview)
    

Book
Empty DataFrame
Columns: []
Index: []


,Class,columns,filename,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0
...,...,...,...,...
977,Book,isbn,Book_cogito-shop.com_September2020.json.gz,3498.0
977,Book,bookformat,Book_cogito-shop.com_September2020.json.gz,3498.0
977,Book,datepublished,Book_cogito-shop.com_September2020.json.gz,3498.0
977,Book,offers,Book_cogito-shop.com_September2020.json.gz,3498.0


CreativeWork
Empty DataFrame
Columns: []
Index: []


,Class,columns,filename,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0
...,...,...,...,...
5517,CreativeWork,author,CreativeWork_manolisheliotis.co.uk_September20...,14.0
5517,CreativeWork,datepublished,CreativeWork_manolisheliotis.co.uk_September20...,14.0
5517,CreativeWork,datemodified,CreativeWork_manolisheliotis.co.uk_September20...,14.0
5517,CreativeWork,publisher,CreativeWork_manolisheliotis.co.uk_September20...,14.0


Event
Empty DataFrame
Columns: []
Index: []


...

,Class,columns,filename,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0
...,...,...,...,...
7831,Recipe,datepublished,Recipe_lesucresale-doumsouhaib.com_September20...,203.0
7831,Recipe,preptime,Recipe_lesucresale-doumsouhaib.com_September20...,203.0
7831,Recipe,cooktime,Recipe_lesucresale-doumsouhaib.com_September20...,203.0
7831,Recipe,totaltime,Recipe_lesucresale-doumsouhaib.com_September20...,203.0


Restaurant
Empty DataFrame
Columns: []
Index: []


,Class,columns,filename,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0
...,...,...,...,...
733,Restaurant,geo,Restaurant_aucklandnz.com_September2020.json.gz,84.0
733,Restaurant,servescuisine,Restaurant_aucklandnz.com_September2020.json.gz,84.0
733,Restaurant,aggregaterating,Restaurant_aucklandnz.com_September2020.json.gz,84.0
733,Restaurant,openinghours,Restaurant_aucklandnz.com_September2020.json.gz,84.0


TVEpisode
Empty DataFrame
Columns: []
Index: []


,Class,columns,filename,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0
...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,431.0
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,431.0
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,431.0
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,63.0


In [43]:
overview

,Class,columns,filename,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0
...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,431.0
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,431.0
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,431.0
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,63.0


In [44]:
overview.to_csv('../../src/data/data/CSV_files/tableoverview.csv', index_label=False)

In [64]:
liste = set(listcolt.loc[index]['listofcolumns'])
display(liste)
overview = overview.explode('columns')
overview = overview[overview['columns'].isin(liste)]
display(overview)

{'episodenumber', 'name', 'partofseries'}

,Class,columns,filename,num_rows,id
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0,Book_name
1,Book,name,Book_virago.co.uk_September2020.json.gz,398.0,Book_name
2,Book,name,Book_borgantiquarian.com_September2020.json.gz,248.0,Book_name
3,Book,name,Book_downpour.com_September2020.json.gz,12194.0,Book_name
4,Book,name,Book_marasworld.com_September2020.json.gz,26.0,Book_name
...,...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,431.0,TVEpisode_name
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,431.0,TVEpisode_episodenumber
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,431.0,TVEpisode_partofseries
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,63.0,TVEpisode_name


In [45]:
overview2 = overview

In [46]:
overview2['id']=overview2['Class']+'_'+overview2['columns']
overview2

,Class,columns,filename,num_rows,id
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,274.0,Book_name
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,274.0,Book_author
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,274.0,Book_publisher
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,274.0,Book_offers
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,274.0,Book_bookedition
...,...,...,...,...,...
208,TVEpisode,name,TVEpisode_foodnetwork.com_September2020.json.gz,431.0,TVEpisode_name
208,TVEpisode,episodenumber,TVEpisode_foodnetwork.com_September2020.json.gz,431.0,TVEpisode_episodenumber
208,TVEpisode,partofseries,TVEpisode_foodnetwork.com_September2020.json.gz,431.0,TVEpisode_partofseries
209,TVEpisode,name,TVEpisode_vavivov.com_September2020.json.gz,63.0,TVEpisode_name


In [61]:
df_stat=pd.concat([overview2,pd.get_dummies(overview2['id'])], axis=1)
sum = df_stat.groupby('filename').sum()
sum =sum.drop(columns={'num_rows'})
df_stat = df_stat.groupby('filename').agg({'Class':'first','num_rows':'first'})
df_stat = df_stat.merge(sum, how='inner', on='filename')
df_stat#.drop(columns={'num_rows_y'})

,Class,num_rows,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
filename,,,,,,,,,,,,,,,,,,,,,
Book_12min.com_September2020.json.gz,Book,366.0,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Book_19shuwu.com_September2020.json.gz,Book,250.0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Book_1carpetcleaning.co.uk_September2020.json.gz,Book,1004.0,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Book_2014brazil.co.uk_September2020.json.gz,Book,387.0,1,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Book_24symbols.com_September2020.json.gz,Book,5171.0,0,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TVEpisode_yesmoviess.com_September2020.json.gz,TVEpisode,37.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
TVEpisode_yidio.com_September2020.json.gz,TVEpisode,12606.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
TVEpisode_zazangels.com_September2020.json.gz,TVEpisode,38.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [62]:
df_stat.to_csv('../../src/data/data/CSV_files/filestats_over_all_columns.csv')

In [66]:
df = pd.read_csv('../../src/data/data/CSV_files/filestats_over_all_columns.csv')
df

,filename,Class,num_rows,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book_12min.com_September2020.json.gz,Book,366.0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Book_19shuwu.com_September2020.json.gz,Book,250.0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Book_1carpetcleaning.co.uk_September2020.json.gz,Book,1004.0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Book_2014brazil.co.uk_September2020.json.gz,Book,387.0,1,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Book_24symbols.com_September2020.json.gz,Book,5171.0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79313,TVEpisode_yesmoviess.com_September2020.json.gz,TVEpisode,37.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
79314,TVEpisode_yidio.com_September2020.json.gz,TVEpisode,12606.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
79315,TVEpisode_zazangels.com_September2020.json.gz,TVEpisode,38.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
79316,TVEpisode_zebrahead.org_September2020.json.gz,TVEpisode,80.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [82]:
df[df['num_rows']<=10].groupby('Class').count()

,filename,num_rows,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
Class,,,,,,,,,,,,,,,,,,,,,
CreativeWork,353,353,353,353,353,353,353,353,353,353,...,353,353,353,353,353,353,353,353,353,353
MusicRecording,6440,6440,6440,6440,6440,6440,6440,6440,6440,6440,...,6440,6440,6440,6440,6440,6440,6440,6440,6440,6440


In [80]:
set(df.columns)

{'Book_aggregaterating',
 'Book_author',
 'Book_bookedition',
 'Book_bookformat',
 'Book_datepublished',
 'Book_genre',
 'Book_inlanguage',
 'Book_isbn',
 'Book_name',
 'Book_numberofpages',
 'Book_offers',
 'Book_publisher',
 'Class',
 'CreativeWork_aggregaterating',
 'CreativeWork_alternativeheadline',
 'CreativeWork_articlebody',
 'CreativeWork_author',
 'CreativeWork_commentcount',
 'CreativeWork_copyrightholder',
 'CreativeWork_copyrightyear',
 'CreativeWork_creator',
 'CreativeWork_datecreated',
 'CreativeWork_datemodified',
 'CreativeWork_datepublished',
 'CreativeWork_genre',
 'CreativeWork_headline',
 'CreativeWork_inlanguage',
 'CreativeWork_interactioncount',
 'CreativeWork_interactionstatistic',
 'CreativeWork_interactiontype',
 'CreativeWork_keywords',
 'CreativeWork_mainentityofpage',
 'CreativeWork_name',
 'CreativeWork_offers',
 'CreativeWork_publisher',
 'CreativeWork_version',
 'Event_doortime',
 'Event_duration',
 'Event_enddate',
 'Event_eventattendancemode',
 'Even